<a href="https://colab.research.google.com/github/parthag1201/MAF_Automation_POCs/blob/main/Customer_Outreach_Campaign_Crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Env and Agent setup

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai

In [2]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
from crewai import Agent, Task, Crew

In [24]:
import os
from google.colab import userdata # For API Secret
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

os.environ["GEMINI_MODEL_NAME"] = 'gemini-2.5-flash'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [6]:
# Agents
from langchain_community.llms import HuggingFaceHub # Import necessary class if using Hugging Face models
from langchain_google_genai import ChatGoogleGenerativeAI # Import the correct class for Gemini

# Initialize the Gemini LLM
os.environ['GOOGLE_API_KEY']=userdata.get('Gemini_ai_api')
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at Amazon, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm = gemini_llm
)

lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of Amazon's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm = gemini_llm
)

In [7]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [8]:
# Instantiate tools
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [ ]:
# print(search_tool.run(search_query="Amazon.com"))

#  Building Custom Crew.ai tool

In [11]:
from crewai_tools import BaseTool

In [26]:
class SentimentAnalysisTool(BaseTool) :
  name: str = "Sentiment Analysis Tool"
  description: str = ("Analyzes the sentiment of text "
  "to ensure positive and engaging communication")

  def _run(self, text: str) -> str:
    # Custom tool code
    return "positive"

In [27]:
sentiment_analysis_tool = SentimentAnalysisTool()

# Tasks definition

In [28]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [29]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	# memory=True
)

In [ ]:
inputs = {
    "lead_name": "Amazon",
    "industry": "Online Shopping Platform",
    "key_decision_maker": "Jeff Bezos",
    "position": "CEO",
    "milestone": "Product Launch (Amazon Alexa)"
}

result = crew.kickoff(inputs=inputs)

In [32]:
from IPython.display import Markdown
Markdown(result)

Here is a series of personalized email drafts for an outreach campaign aimed at Jeff Bezos, Executive Chair of Amazon:

---

**Email 1: Initial Outreach**

**Subject: Elevating Amazon's AI Vision: A Strategic Partnership Opportunity**

Dear Mr. Bezos,

As the visionary founder and Executive Chair of Amazon, your relentless pursuit of innovation and customer obsession continues to redefine industries. We've closely followed Amazon's incredible journey, particularly your pioneering work in artificial intelligence, exemplified by the transformative impact of products like Amazon Alexa.

At [Your Company Name], we specialize in advanced AI/ML solutions designed to amplify the capabilities of market leaders. We believe our expertise in optimizing data insights from vast user interactions and streamlining complex AI model deployment could offer significant strategic value as Amazon continues to push the boundaries of AI-driven customer experiences and operational excellence.

We are particularly keen to explore how our solutions could support Amazon's ongoing advancements in AI, ensuring even greater personalization, efficiency, and scale across your diverse ecosystem.

Would you be open to a brief, strategic conversation with my team? We'd be honored to share how our approach aligns with Amazon's long-term vision for innovation and customer-centricity.

Warm regards,

[Your Name]
Lead Sales Representative
[Your Company Name]

---

**Email 2: Follow-up / Deeper Dive**

**Subject: Following Up: Further Enhancing Amazon's AI Ecosystem**

Dear Mr. Bezos,

Hope this email finds you well.

Following up on my previous message, I wanted to reiterate our profound respect for Amazon's pioneering spirit, especially in leveraging AI to create unparalleled customer experiences. The continuous evolution of intelligent systems like Alexa truly embodies Amazon's commitment to long-term thinking and innovation.

Our advanced AI/ML solutions are specifically designed to complement and enhance large-scale AI initiatives, focusing on areas such as predictive analytics for user behavior, optimizing data processing from device interactions, and automating complex AI workflows. We envision our capabilities as a strategic asset for Amazon, further refining the personalization and efficiency that define your customer-centric approach.

Imagine even more seamless integration, richer insights from user data, and accelerated development cycles for your next generation of AI-driven services. Our goal is to empower Amazon to achieve even greater breakthroughs.

Would a brief 15-minute call with my team be a convenient way to explore these possibilities? We are confident we can demonstrate tangible value aligned with Amazon's strategic priorities.

Warm regards,

[Your Name]
Lead Sales Representative
[Your Company Name]

---

**Email 3: Value Reinforcement / Call to Action**

**Subject: Accelerating Amazon's AI Advantage: A Proven Path to Enhanced Value**

Dear Mr. Bezos,

I hope this message finds you well and your focus on Amazon's future innovations remains as sharp as ever.

In our previous communications, we've highlighted how [Your Company Name]'s advanced AI/ML solutions are designed to support and amplify the strategic objectives of global leaders like Amazon. Given your profound impact on shaping Amazon's customer-centric culture and long-term vision, we believe our expertise can significantly contribute to your ongoing advancements in AI, including the continuous evolution of platforms like Alexa.

Our solutions are built for scale and efficiency, offering tangible benefits such as:
*   **Enhanced Customer Personalization:** Driving deeper engagement through more intelligent and proactive user experiences.
*   **Optimized Data Insights:** Transforming vast datasets from devices and interactions into actionable intelligence.
*   **Streamlined AI Operations:** Automating complex processes to accelerate innovation and reduce operational costs.

These capabilities directly align with Amazon's commitment to operational excellence, customer obsession, and sustainable growth. We have a proven track record of delivering measurable ROI for enterprises operating at Amazon's scale.

We understand your time is invaluable. Would it be possible to schedule a concise 20-minute discussion with a member of your leadership team who oversees AI strategy or customer experience initiatives? We are prepared to present a tailored overview demonstrating how our partnership can unlock new levels of value for Amazon.

Thank you for considering this opportunity.

Sincerely,

[Your Name]
Lead Sales Representative
[Your Company Name]